# III Assignment - Routing Algorithms
## Bidirectional Dijkstra

---
## Dependencies
Necessaries imports

In [1]:
from ipynb.fs.full.exercise1 import initialize_graph, min_distance
from ipynb.fs.full.exercise2a import contract, build_predecessors
from heap import binheap

---
# Reverse Graph
Bidirectional Dijkstra works on a graph and on his transpose graph. The following function takes a graph and it returns his transpose.

In [2]:
def transpose_graph(graph):
    transpose_graph = dict()

    # copy graph in reverse graph but with adjacenct list empty
    for start_node, value in graph.items():
        transpose_graph.setdefault(start_node, {'adjacency_list': dict(), 'importance': value.get('importance')})

    # fill the adjacency lists
    for start_node, value in graph.items():
        for adjacent, distance in graph.get(start_node).get('adjacency_list').items():
            transpose_graph.get(adjacent).get(
                'adjacency_list').setdefault(start_node, distance)

    return transpose_graph


---
## Bidirectional Dijkstra
The following function performs the bidirectional version of Dijkstra

In [12]:
def bidirectional_dijkstra(graph, source, destination):

    # build the graphs for the forward search and the backward search
    graph_fw = graph
    graph_bw = transpose_graph(graph)
    
    # initialization of the graphs
    initialize_graph(graph_fw, source)
    initialize_graph(graph_bw, destination)
    
    # every search has to have its own binary heap queue 
    queue_fw = binheap([[key, attributes.get('distance')] for key, attributes in graph_fw.items()], total_order = min_distance)
    queue_bw = binheap([[key, attributes.get('distance')] for key, attributes in graph_bw.items()], total_order = min_distance)
    
    common_node = None
    
    iteration = 1

    current_node_fw, _ = queue_fw.remove_minimum()
    current_node_bw, _ = queue_bw.remove_minimum()
    
    while not queue_fw.is_empty() and not queue_bw.is_empty():
        
        print('\nIteration Number', iteration)
        iteration = iteration + 1
        
        # relax forward and backward
        relax(graph_fw, current_node_fw, queue_fw)
        relax(graph_bw, current_node_bw, queue_bw)

        # forward: the node in which I will step into is a valid node?
        if graph_fw.get(queue_fw.get_minimum()[0]).get('importance') <= graph_fw.get(current_node_fw).get('importance'):
            contract(graph_fw, queue_fw.get_minimum()[0])
            print('[forward  search]: removed node', queue_fw.remove_minimum()[0])
        else:
            current_node_fw, _ = queue_fw.remove_minimum()
            graph_fw.get(current_node_fw)['visited'] = True
            print('[forward  search]: stepped into', current_node_fw)
            if graph_bw.get(current_node_fw) is not None and graph_bw.get(current_node_fw).get('visited') is True:
                print('\n[forward  search]: the node', current_node_fw, 'has been already visited by backward search')
                common_node = current_node_fw
                break;
                                     
        # backward: the node in which I will step into is a valid node?
        if graph_bw.get(queue_bw.get_minimum()[0]).get('importance') <= graph_bw.get(current_node_bw).get('importance'):
            contract(graph_bw, queue_bw.get_minimum()[0])
            print('[backward search]: removed node', queue_bw.remove_minimum()[0])
        else:
            current_node_bw, _ = queue_bw.remove_minimum()
            graph_bw.get(current_node_bw)['visited'] = True
            print('[backward search]: stepped into', current_node_bw)
            if graph_fw.get(current_node_bw) is not None and graph_fw.get(current_node_bw).get('visited') is True:
                print('\n[backward search]: the node', current_node_bw, 'has been already visited by forward search')
                common_node = current_node_bw
                break;

    if common_node is not None:
        path_fw = build_predecessors(graph_fw, common_node)
        path_bw = build_predecessors(graph_bw, common_node)
        path_bw.reverse()
        path_bw.remove(common_node)
        return graph_fw.get(common_node).get('distance') + graph_bw.get(common_node).get('distance'), path_fw + path_bw
    else:
        return -1, None

In [15]:
if __name__ == '__main__':
    graph = {}
    graph.setdefault('A', {'adjacency_list': {'B': 3, 'E': 5}, 'importance': 5})
    graph.setdefault('B', {'adjacency_list': {'C': 3},         'importance': 2})
    graph.setdefault('C', {'adjacency_list': {'D': 3},         'importance': 2})
    graph.setdefault('D', {'adjacency_list': {},               'importance': 5})
    graph.setdefault('E', {'adjacency_list': {'D': 5},         'importance': 8})

    graph2 = {}
    graph2.setdefault('A', {'adjacency_list': {'B': 3,'E': 5},  'importance': 1})
    graph2.setdefault('B', {'adjacency_list': {},               'importance': 1})
    graph2.setdefault('C', {'adjacency_list': {'B': 3},         'importance': 1})
    graph2.setdefault('D', {'adjacency_list': {'C': 3},         'importance': 1})
    graph2.setdefault('E', {'adjacency_list': {'D': 5},         'importance': 3})

    graph3 = {}
    graph3.setdefault('A', {'adjacency_list': {'B': 4},                 'importance': 20})
    graph3.setdefault('B', {'adjacency_list': {'C': 11, 'D': 9},        'importance': 7})
    graph3.setdefault('C', {'adjacency_list': {'A': 8},                 'importance': 55})
    graph3.setdefault('D', {'adjacency_list': {'E': 2, 'F': 6, 'C': 7}, 'importance': 1})
    graph3.setdefault('E', {'adjacency_list': {'B': 8, 'G': 7, 'H': 4}, 'importance': 1})
    graph3.setdefault('F', {'adjacency_list': {'C': 1, 'E': 5},         'importance': 1})
    graph3.setdefault('G', {'adjacency_list': {'H': 14, 'I': 9},        'importance': 5})
    graph3.setdefault('H', {'adjacency_list': {'F': 2, 'I': 10},        'importance': 7})
    graph3.setdefault('I', {'adjacency_list': {},                       'importance': 20})
    
    print(bidirectional_dijkstra(graph3, 'A', 'I'))
    #print(transpose_graph(graph))



Iteration Number 1
[forward  search]: removed node B
[backward search]: removed node G

Iteration Number 2
[forward  search]: removed node D
[backward search]: removed node H

Iteration Number 3
[forward  search]: removed node E
[backward search]: removed node E

Iteration Number 4
[forward  search]: stepped into C
[backward search]: removed node D

Iteration Number 5
[forward  search]: removed node H
[backward search]: removed node F

Iteration Number 6
[forward  search]: removed node F
[backward search]: removed node B

Iteration Number 7
[forward  search]: removed node G
[backward search]: removed node A

Iteration Number 8
[forward  search]: removed node I
[backward search]: stepped into C

[backward search]: the node C has been already visited by forward search
(52, ['A', 'C', 'I'])
